## Algorithm to map the neighbourhoods in Toronto

The datset is not readily availaable, hence needs to be scraped.

## Data Collection and Pre-processing

### Scraping a wikipedia page using "BeautifulSoup"

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(page.content, 'html.parser')

In [4]:
table=soup.find("tbody")
rows=table.select("tr")
row=[r.get_text() for r in rows]

#### Clean the dataframe

In [21]:
df=pd.DataFrame(row)
df1=df[0].str.split('\n',expand=True)
df2=df1.rename(columns=df1.iloc[0])
df3=df2.drop(df2.index[0])
df3.head()

,,Postcode,Borough,Neighbourhood,
1,,M1A,Not assigned,Not assigned,
2,,M2A,Not assigned,Not assigned,
3,,M3A,North York,Parkwoods,
4,,M4A,North York,Victoria Village,
5,,M5A,Downtown Toronto,Harbourfront,


#### Delete cells where borough is not assigned

In [27]:
df4=df3[df3.Borough != 'Not assigned']
df4

,,Postcode,Borough,Neighbourhood,
3,,M3A,North York,Parkwoods,
4,,M4A,North York,Victoria Village,
5,,M5A,Downtown Toronto,Harbourfront,
6,,M6A,North York,Lawrence Heights,
7,,M6A,North York,Lawrence Manor,
...,...,...,...,...,...
282,,M8Z,Etobicoke,Kingsway Park South West,
283,,M8Z,Etobicoke,Mimico NW,
284,,M8Z,Etobicoke,The Queensway West,
285,,M8Z,Etobicoke,Royal York South West,


In [33]:
#df = pd.merge(df6, df7, on='Postcode')
df5 = df4.groupby(['Postcode', 'Borough'], sort = False).agg(','.join)
df5.reset_index(inplace=True)
df5


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
...,...,...,...
98,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
101,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout..."


In [39]:
df5["Neighbourhood"].replace("Not assigned","Queen's Park",inplace=True)
df5.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


In [41]:
df5.shape

(103, 3)

## Mapping

In [45]:
# install geopy & folium
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\91984\Anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          92 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0




geographiclib-1.50   | 34 KB     |            |   0% 
geographiclib-1.50   | 34 KB     | ####7      |  47% 
geographiclib-1.50   | 34 KB     | ########## | 100% 

geopy-1.21.0         | 58 KB     |            |   0% 
geopy-1.21.0         | 58 K

In [47]:
url = "http://cocl.us/Geospatial_data"
df7 = pd.read_csv(url)
df7.rename(columns={'Postal Code': 'Postcode'}, inplace=True) # chane the first column's name to be as the first dataframe
df7.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### After generating the geospatial coordinates, create a dataframe merging toronto neighborhoods with the corresponding latitude and longitude

In [48]:
df6 = pd.merge(df5, df7, on='Postcode')
df6.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


## Visualising Toronto Neighbourhoods : Clustering and segmentation

### Choose Toronto boroughs

In [60]:
Toronto=df6[df6['Borough'].str.contains('Toronto')]
Toronto.reset_index(inplace=True)
Toronto.head()

,index,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
2,9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
3,15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [90]:
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
address = 'Toronto'
geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

Toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(Toronto['Latitude'], Toronto['Longitude'], 
                                           Toronto['Borough'], Toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Toronto_map)  
    
Toronto_map

# Segmentation

In [54]:
CLIENT_ID = 'OQYWSZCAO25AV4Z3KPAZQXOKTGGAIQBJHJXSLLXOFG3I0DRW' # your Foursquare ID
CLIENT_SECRET = 'LRONLXGCQQFSHIBIBN4Z3X2WT0CCXUZFRREYSXO1G1QGXM4H' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


## Analyse neighbourhood

In [65]:
Toronto.loc[0, 'Neighbourhood']
LIMIT = 5

In [66]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [67]:
Toronto_Venues = getNearbyVenues(names=Toronto['Neighbourhood'],
                                   latitudes=Toronto['Latitude'],
                                   longitudes=Toronto['Longitude']
                                  )

Harbourfront
Queen's Park
Ryerson,Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
The Danforth West,Riverdale
Design Exchange,Toronto Dominion Centre
Brockton,Exhibition Place,Parkdale Village
The Beaches West,India Bazaar
Commerce Court,Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North,Forest Hill West
High Park,The Junction South
North Toronto West
The Annex,North Midtown,Yorkville
Parkdale,Roncesvalles
Davisville
Harbord,University of Toronto
Runnymede,Swansea
Moore Park,Summerhill East
Chinatown,Grange Park,Kensington Market
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown,St. James Town
First Canadian Place,Underground city

In [68]:
Toronto_Venues.shape

(188, 7)

### further analysis

In [70]:
Toronto_onehot = pd.get_dummies(Toronto_Venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] =Toronto_Venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]
Toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Bakery,Bar,...,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tea Room,Tennis Court,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Neighbourhood with top 5 common venues

In [71]:
t = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
t

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Bakery,...,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tea Room,Tennis Court,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant
0,"Adelaide,King,Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.2,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.2
1,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.2,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.2
2,"Brockton,Exhibition Place,Parkdale Village",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.0
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.0
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.0,0.2,0.2,0.2,0.2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.0
5,"Cabbagetown,St. James Town",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.0
6,Central Bay Street,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.0
7,"Chinatown,Grange Park,Kensington Market",0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.2,...,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.0
8,Christie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.0
9,Church and Wellesley,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.2,0.00,0.0


In [72]:
num_top_venues = 5

for hood in t['Neighborhood']:
    print("----"+hood+"----")
    temp = t[t['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                           venue  freq
0  Vegetarian / Vegan Restaurant   0.2
1                          Plaza   0.2
2                          Hotel   0.2
3                     Steakhouse   0.2
4                   Concert Hall   0.2


----Berczy Park----
                           venue  freq
0  Vegetarian / Vegan Restaurant   0.2
1                   Liquor Store   0.2
2                    Coffee Shop   0.2
3                     Steakhouse   0.2
4                   Concert Hall   0.2


----Brockton,Exhibition Place,Parkdale Village----
          venue  freq
0   Coffee Shop   0.4
1           Gym   0.2
2          Café   0.2
3           Bar   0.2
4  Liquor Store   0.0


----Business Reply Mail Processing Centre 969 Eastern----
            venue  freq
0      Comic Shop   0.2
1         Brewery   0.2
2     Pizza Place   0.2
3   Burrito Place   0.2
4  Farmers Market   0.2


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lan

### Convert to a dataframe

function to sort in descending order

In [73]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

create the dataframe

In [76]:
import numpy as np

In [78]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = t['Neighborhood']

for ind in np.arange(t.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(t.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Vegetarian / Vegan Restaurant,Concert Hall,Plaza,Hotel,Steakhouse,Diner,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio
1,Berczy Park,Vegetarian / Vegan Restaurant,Concert Hall,Liquor Store,Steakhouse,Coffee Shop,Dessert Shop,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio
2,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Bar,Café,Gym,Food & Drink Shop,Food,Concert Hall,Cosmetics Shop,Gastropub,Creperie
3,Business Reply Mail Processing Centre 969 Eastern,Pizza Place,Farmers Market,Burrito Place,Brewery,Comic Shop,Food,Department Store,Concert Hall,Cosmetics Shop,Creperie
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport,Airport Food Court,Airport Lounge,Airport Terminal,Harbor / Marina,Vegetarian / Vegan Restaurant,Eastern European Restaurant,Creperie,Cuban Restaurant,Dance Studio


### K means Clustering

In [79]:
# set number of clusters
kclusters = 5

t_cluster = t.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(t_cluster)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 3, 3, 2, 2, 4, 3, 2, 3, 2])

In [82]:


t_merged = Toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
t_merged = t_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

t_merged.head() # check the last columns!

,index,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,3,Spa,Gym / Fitness Center,Bakery,Restaurant,Coffee Shop,Diner,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant
1,4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,3,Gym,Italian Restaurant,Portuguese Restaurant,Park,Coffee Shop,Fish Market,Dance Studio,Comic Shop,Concert Hall,Cosmetics Shop
2,9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,1,Pizza Place,Clothing Store,Comic Shop,Plaza,Café,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio
3,15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,4,Gym,Italian Restaurant,Creperie,Restaurant,Japanese Restaurant,Fish Market,Department Store,Concert Hall,Cosmetics Shop,Garden
4,19,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Trail,Health Food Store,Pub,Vegetarian / Vegan Restaurant,Diner,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio


## Visualise

In [84]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(t_merged['Latitude'], t_merged['Longitude'],t_merged['Neighbourhood'], t_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining Clusters

#### Cluster 1

In [85]:
t_merged.loc[t_merged['Cluster Labels'] == 0, t_merged.columns[[1] + list(range(5,t_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,M4T,-79.383160,0,Park,Tennis Court,Playground,Restaurant,Department Store,Coffee Shop,Comic Shop,Concert Hall,Cosmetics Shop,Creperie
33,M4W,-79.377529,0,Park,Trail,Playground,Vegetarian / Vegan Restaurant,Dessert Shop,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant


We infer, cluster 0 refers to being near Parks and Playgrounds

In [86]:
t_merged.loc[t_merged['Cluster Labels'] == 1, t_merged.columns[[1] + list(range(5,t_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5B,-79.378937,1,Pizza Place,Clothing Store,Comic Shop,Plaza,Café,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio
24,M5R,-79.405678,1,Vegetarian / Vegan Restaurant,Café,Middle Eastern Restaurant,American Restaurant,Indian Restaurant,Diner,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio
25,M6R,-79.456325,1,Dog Run,Movie Theater,Cuban Restaurant,Eastern European Restaurant,Dessert Shop,Diner,Concert Hall,Cosmetics Shop,Creperie,Dance Studio
26,M4S,-79.388790,1,Dessert Shop,Pizza Place,Café,Indian Restaurant,Diner,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio


In [87]:
t_merged.loc[t_merged['Cluster Labels'] == 3, t_merged.columns[[1] + list(range(5,t_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,-79.360636,3,Spa,Gym / Fitness Center,Bakery,Restaurant,Coffee Shop,Diner,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant
1,M7A,-79.389494,3,Gym,Italian Restaurant,Portuguese Restaurant,Park,Coffee Shop,Fish Market,Dance Studio,Comic Shop,Concert Hall,Cosmetics Shop
5,M5E,-79.373306,3,Vegetarian / Vegan Restaurant,Concert Hall,Liquor Store,Steakhouse,Coffee Shop,Dessert Shop,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio
6,M5G,-79.387383,3,Coffee Shop,Gastropub,Park,Garden,Food & Drink Shop,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant
7,M6G,-79.422564,3,Coffee Shop,Italian Restaurant,Café,Diner,Grocery Store,Food,Department Store,Concert Hall,Cosmetics Shop,Gastropub
13,M5K,-79.381576,3,Gym,Pub,Hotel,Restaurant,Coffee Shop,Dessert Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant
14,M6K,-79.428191,3,Coffee Shop,Bar,Café,Gym,Food & Drink Shop,Food,Concert Hall,Cosmetics Shop,Gastropub,Creperie
15,M4L,-79.315572,3,Gym,Fish & Chips Shop,Ice Cream Shop,Burger Joint,Italian Restaurant,Diner,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio
16,M5L,-79.379817,3,Gym,Coffee Shop,Café,Pub,Restaurant,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio
17,M4M,-79.340923,3,Bookstore,Fish Market,Ice Cream Shop,Coffee Shop,Sandwich Place,Diner,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant


In [88]:
t_merged.loc[t_merged['Cluster Labels'] == 4, t_merged.columns[[1] + list(range(5,t_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,M5C,-79.375418,4,Gym,Italian Restaurant,Creperie,Restaurant,Japanese Restaurant,Fish Market,Department Store,Concert Hall,Cosmetics Shop,Garden
12,M4K,-79.352188,4,Greek Restaurant,Cosmetics Shop,Italian Restaurant,Dog Run,Concert Hall,Creperie,Cuban Restaurant,Dance Studio,Department Store,Dessert Shop
27,M5S,-79.400049,4,Bookstore,Japanese Restaurant,Restaurant,Dessert Shop,Italian Restaurant,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Department Store
35,M4X,-79.367675,4,Japanese Restaurant,Diner,Restaurant,Indian Restaurant,Italian Restaurant,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio
